# Create fine-tuning data

In [1]:
import pandas as pd
import json 

In [2]:
ls

Codebook for extractive summary.docx
CreateFineTuningData.ipynb
InterAnnotator2.ipynb
SynopsisFinder.py
T. Underwood Fund Balance Summary_08312025_nr.xlsx
all_synopses.csv
annotations/
annotations2/
avery_synopses.xlsx
lab_annotated.csv
longer_synopses.csv
longer_synopses.json
new_gutenberg_synopses.json
no_synopses_sample.csv
paragraphs-extracted.json
selected_long_synopses.csv
selected_longer2_synopses.csv
selected_longer_synopses.csv
selected_new_gutenberg_synopses.csv
selected_synopses.csv
shared_longer_synopses.csv
shared_synopses.csv
some samples.xlsx
synopses.json
synopses_for_annotation.csv
synopses_output_gemini.json
synopsis_unpacker.ipynb
test_cases_for_codebook.csv
test_cases_for_codebook_ted.csv
twelve_test_cases.csv


In [3]:
qiyan_data = json.load(open('synopses_output_gemini.json'))

In [5]:
qiyan_data[0]

{'position': 61,
 'filename': '70203.txt',
 'passage': 'Schubert’s fame rests chiefly on his songs, but in the domain of\ninstrumental music he also did some remarkable work, although it\ncan hardly be said that he fundamentally influenced the art in\nthis direction. He lived (1797-1828) in Vienna at the same time as\nBeethoven, and many of his compositions show distinctly how much\nhe was influenced by the genius of that great musician. Schubert’s\nworks are distinguished by a spontaneous flow of beautiful melody.\nHe possessed, by right of nature, the utmost artistic opulence, but\nunfortunately, owing chiefly to his birth and early circumstances, this\nwas not consolidated by that profounder culture which is necessary for\nthe fullest expression of human capacity. Not that Schubert failed\nto reach a most exalted position, for works like the B♭ Piano Trio,\nthe C major Quintett, that noble fragment the B minor Symphony, and\nespecially his best songs, proclaim him unsurpassed by any

In [6]:
print(qiyan_data[0]['passage'])

Schubert’s fame rests chiefly on his songs, but in the domain of
instrumental music he also did some remarkable work, although it
can hardly be said that he fundamentally influenced the art in
this direction. He lived (1797-1828) in Vienna at the same time as
Beethoven, and many of his compositions show distinctly how much
he was influenced by the genius of that great musician. Schubert’s
works are distinguished by a spontaneous flow of beautiful melody.
He possessed, by right of nature, the utmost artistic opulence, but
unfortunately, owing chiefly to his birth and early circumstances, this
was not consolidated by that profounder culture which is necessary for
the fullest expression of human capacity. Not that Schubert failed
to reach a most exalted position, for works like the B♭ Piano Trio,
the C major Quintett, that noble fragment the B minor Symphony, and
especially his best songs, proclaim him unsurpassed by any musician
of modern times; but out of the very profusion of his ideas

In [7]:
# Function that accepts a string of English text and splits it into sentences
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
def split_into_sentences(text):
    return sent_tokenize(text)

[nltk_data] Downloading package punkt to /Users/tunder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# Go through each element of qiyan_data. For each element, we're going
# to add the model_synopsis_sentence field to the passage field. But
# the position of the added sentence will vary randomly:
#
# 30% cases: beginning
# 40% cases: middle, between two random sentences
# 30% cases: end
import random
for entry in qiyan_data:
    passage = entry['passage']
    model_synopsis_sentence = entry['model_synopsis_sentence']
    sentences = split_into_sentences(passage)
    rand_val = random.random()
    if rand_val < 0.3:
        # beginning
        new_passage = model_synopsis_sentence + " " + passage
    elif rand_val < 0.7:
        # middle
        if len(sentences) > 1:
            insert_pos = random.randint(1, len(sentences)-1)
            new_passage = " ".join(sentences[:insert_pos]) + " " + model_synopsis_sentence + " " + " ".join(sentences[insert_pos:])
        else:
            new_passage = passage + " " + model_synopsis_sentence
    else:
        # end
        new_passage = passage + " " + model_synopsis_sentence
    entry['passage_with_synopsis'] = new_passage


In [9]:
# Randomly copy 40 files from /Users/tunder/workdata/hathifiles/gutenberg_texts
# to /Users/tunder/workdata/hathifiles/new_gutenberg
import shutil
import os
source_dir = '/Users/tunder/workdata/hathifiles/gutenberg_texts'
target_dir = '/Users/tunder/workdata/hathifiles/new_gutenberg'
all_files = os.listdir(source_dir)
selected_files = random.sample(all_files, 40)
for filename in selected_files:
    shutil.copy(os.path.join(source_dir, filename), os.path.join(target_dir, filename))

In [13]:
# Gather negatives from

filelist = ['synopses.json', 'longer_synopses.json', 'fullest_gutenberg_synopses.json']

# for each file, load the json data
# and add entries with synopsis_present = 'n' to negatives list
# unless the first 50 characters of the passage are already in the
# prefix set
prefixset = set()
negatives = []
for file in filelist:
    data = json.load(open(file))
    for entry in data['results']:
        if entry['synopsis_present'] == 'y':
            continue
        prefix = entry['passage'][:50]
        if prefix not in prefixset:
            entry['synopsis_present'] = 'n'
            negatives.append(entry)
            prefixset.add(prefix)
        else:
            print("Duplicate prefix, skipping")

Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
Duplicate prefix, skipping
D

In [14]:
len(negatives)

788

In [12]:
data['results'][0]

{'filename': '5786.txt',
 'passage': "When we began that ascent, we could see a microscopic chalet perched\naway up against heaven on what seemed to be the highest mountain near\nus. It was on our right, across the narrow head of the valley. But when\nwe got up abreast it on its own level, mountains were towering high\nabove on every hand, and we saw that its altitude was just about that of\nthe little Gasternthal which we had visited the evening before. Still it\nseemed a long way up in the air, in that waste and lonely wilderness of\nrocks. It had an unfenced grass-plot in front of it which seemed about\nas big as a billiard-table, and this grass-plot slanted so sharply\ndownward, and was so brief, and ended so exceedingly soon at the verge\nof the absolute precipice, that it was a shuddery thing to think of a\nperson's venturing to trust his foot on an incline so situated at all.\nSuppose a man stepped on an orange peel in that yard; there would be\nnothing for him to seize; nothing

In [19]:
manual_data = pd.read_csv('annotations2/manual_data_combined.csv')
manual_dictionaries = []
for index, row in manual_data.iterrows():
    entry = {
        'passage': row['passage'],
        'filename': row['filename'],
        'synopsis_present': row['has_synopsis'],
        'synopsis': row['synopsis_sentence']
    }
    manual_dictionaries.append(entry)

In [20]:
def create_system_prompt() -> str:
    """
    Create prompt for summary generation
    """  
    prompt = f"""You are sorting paragraphs into two groups: those that contain a sentence that previews or summarizes the whole paragraph, and those that lack single-sentence synopsis.

Most paragraphs do not contain a synopsis sentence, and if one does not already exist you should not create one. For instance, consider the following paragraph:

This may clearly be seen by taking a very small quantity of such a substance as chlorate of potash. If a crystal of this is examined under a magnifying glass till its crystalline form and structure are familiar, and it is then placed in a test-tube and gently heated, cleavage will at once be evident. With a little crackling, the chlorate splits itself into many crystals along its chief lines of cleavage (called the cleavage planes), every one of which crystals showing under the microscope the identical form and characteristics of the larger crystal from which it came.

No single sentence there adequately summarizes the whole. If you determine that the paragraph does not contain a good synopsis, respond ONLY with the following JSON object:

{{"synopsis_present": "n", "synopsis": ""}}

If there is a synopsis, you will answer ONLY with a valid JSON object in exactly this format:

{{"synopsis_present": "y", "synopsis": <single verbatim sentence from the text>}}

For instance, given the following paragraph:

First, I went to the wrong classroom for math. I was sitting in the class, surrounded by people taking notes and paying attention to how to do equations, which would have been okay if I was supposed to be in an algebra class. In reality, I was supposed to be in geometry, and when I discovered my error, I had already missed the first twenty minutes of a one-hour class. When I got to the correct class, all twenty-five students turned and looked at me as the teacher said, "You're late." That would have been bad enough, but in my next class my history teacher spoke so fast I could not follow most of what they said. The only thing I did hear was that we were having a quiz tomorrow over today’s lecture. My day seemed to be going better during botany class, that is, until we visited the lab. I had a sneezing fit because of one of the plants in the lab and had to leave the room. When I finally finished my classes for the day, I discovered I had locked my keys in the car and had to wait for my brother to bring another set. My first day of college was a disaster. 

The correct response would be:

{{"synopsis_present": "y", "synopsis": "My first day of college was a disaster."}}"""
    
    return prompt

In [21]:
qiyan_data[0]

{'position': 61,
 'filename': '70203.txt',
 'passage': 'Schubert’s fame rests chiefly on his songs, but in the domain of\ninstrumental music he also did some remarkable work, although it\ncan hardly be said that he fundamentally influenced the art in\nthis direction. He lived (1797-1828) in Vienna at the same time as\nBeethoven, and many of his compositions show distinctly how much\nhe was influenced by the genius of that great musician. Schubert’s\nworks are distinguished by a spontaneous flow of beautiful melody.\nHe possessed, by right of nature, the utmost artistic opulence, but\nunfortunately, owing chiefly to his birth and early circumstances, this\nwas not consolidated by that profounder culture which is necessary for\nthe fullest expression of human capacity. Not that Schubert failed\nto reach a most exalted position, for works like the B♭ Piano Trio,\nthe C major Quintett, that noble fragment the B minor Symphony, and\nespecially his best songs, proclaim him unsurpassed by any

In [22]:
negatives[0]

{'filename': '909.txt',
 'passage': "Every day we experienced recent mischiefs. The barbarous savage nations\nof Shawanese, Cherokees, Wyandots, Tawas, Delawares, and several others\nnear Detroit, united in a war against us, and assembled their choicest\nwarriors at old Chelicothe, to go on the expedition, in order to\ndestroy us, and entirely depopulate the country. Their savage minds were\ninflamed to mischief by two abandoned men, Captains McKee and Girty.\nThese led them to execute every diabolical scheme; and, on the fifteenth\nday of August, commanded a party of Indians and Canadians, of about five\nhundred in number, against Briant's station, five miles from Lexington.\nWithout demanding a surrender, they furiously assaulted the garrison,\nwhich was happily prepared to oppose them; and, after they had expended\nmuch ammunition in vain, and killed the cattle round the fort, not being\nlikely to make themselves masters of this place, they raised the siege,\nand departed in the mor

In [23]:
manual_dictionaries[0]

{'passage': '“It’s so nice, Bertie, talking to somebody who really takes a sensible\nview about this man Glossop. Mother says he’s a good chap, which is\nsimply absurd. Anybody can see that he’s absolutely impossible. He’s\nconceited and opinionative and argues all the time, even when he knows\nperfectly well that he’s talking through his hat, and he smokes too\nmuch and eats too much and drinks too much, and I don’t like the colour\nof his hair. Not that he’ll have any hair in a year or two, because\nhe’s pretty thin on the top already, and before he knows where he is\nhe’ll be as bald as an egg, and he’s the last man who can afford to\ngo bald. And I think it’s simply disgusting, the way he gorges all\nthe time. Do you know, I found him in the larder at one o’clock this\nmorning, absolutely wallowing in a steak-and-kidney pie? There was\nhardly any of it left. And you remember what an enormous dinner he\nhad. Quite disgusting, I call it. But I can’t stop out here all night,\ntalking 

In [24]:
# We have three sources of data that need to be combined into a single master dataset.
# But the three sources are in different formats, so we need to standardize them first.
# Also we need to add a system prompt to each entry.
# Finally, in 10% of the negatives, we will change the position of a single sentence.

def move_a_sentence_randomly(passage: str) -> str:
    sentences = split_into_sentences(passage)
    if len(sentences) < 2:
        return passage
    sentence_to_move = random.choice(sentences)
    sentences.remove(sentence_to_move)
    insert_pos = random.randint(0, len(sentences))
    new_sentences = sentences[:insert_pos] + [sentence_to_move] + sentences[insert_pos:]
    return " ".join(new_sentences)

system_prompt = create_system_prompt()

def message_format(user, assistant):
    return {"messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user},
        {"role": "assistant", "content": assistant}
    ]}

master_dataset = []
for entry in qiyan_data:
    user_message = entry['passage_with_synopsis']
    assistant_message = f"{{\"synopsis_present\": \"y\", \"synopsis\": \"{entry['model_synopsis_sentence']}\"}}"
    standardized_entry = message_format(user_message, assistant_message)    
    master_dataset.append(standardized_entry)

for entry in negatives:
    passage = entry['passage']
    if random.random() < 0.1:
        passage = move_a_sentence_randomly(passage)
    user_message = passage
    assistant_message = f"{{\"synopsis_present\": \"n\", \"synopsis\": \"\"}}"
    standardized_entry = message_format(user_message, assistant_message)    
    master_dataset.append(standardized_entry)

for entry in manual_dictionaries:
    passage = entry['passage'] 
    user_message = passage
    if entry['synopsis_present'] == 'y':
        assistant_message = f"{{\"synopsis_present\": \"y\", \"synopsis\": \"{entry['synopsis']}\"}}"
    else:
        assistant_message = f"{{\"synopsis_present\": \"n\", \"synopsis\": \"\"}}"
    standardized_entry = message_format(user_message, assistant_message)    
    master_dataset.append(standardized_entry)

len(master_dataset)

973

In [25]:
# Shuffle master_dataset and write to jsonl file
random.shuffle(master_dataset)
with open('fine_tuning_data_synopsis_detection.jsonl', 'w') as f:
    for entry in master_dataset:
        f.write(json.dumps(entry) + '\n')